## <right>Binôme 
* Saliou Barry
* Zhile Zhang
</right>
***

In [ ]:
# !pip install -q lxml

import bs4
import lxml
import pandas
import urllib

from urllib import request

## Exercice 1 : Scrap a table on a wikipedia page

We would like to display on a map the location of Summer Olympic Games since 1896. We will use a [Wikipedia page](https://fr.wikipedia.org/wiki/Jeux_olympiques) to scrap the associated table.
Below is the code to extract the content of the page using `request` and display its title using `BeautifulSoup`.

In [ ]:
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

request_text = request.urlopen(jo).read()
print(request_text[:1000])
page = bs4.BeautifulSoup(request_text, "lxml")
print(page.find("title"))

# Web Scrapping in Python


**TODO pour moi avant le TME**

1.   Faire la version étudiant en enlevant les réponses. Ne garder qu'un code générique pour la carte qu'ils devront adapter



The first objective of this notebook is to discover the `request` and `BeautifulSoup` libraries to crawl a table on a Wikitable page, build a dataframe, and create a map.

*   `request` and [urllib](https://docs.python.org/3/library/urllib.html#module-urllib) for requestion REST API
*   [BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to inspect webpages

Please note that this part serves as an initiation of web scrapping but you will need to learn by yourself to make the project. It is inspired from notebooks published by Galiana, Lino. 2023. Python Pour La Data Science. https://doi.org/10.5281/zenodo.8229676. Exercices target different sources of information, the code needs to be adapted.











Our objective here is to extract the different information in the first table "Jeux olympiques d'été" and to build a data frame.

To proceed, you will have to follow these steps:
*   Find the table
*   Collect each line of the table
*   Retrieve the different columns and transform them into text format. Also, use `strip` to format the value into a proper text format (e.g., without useless spaces). Store these lines (formated as a table of columns) in a table.

**Warning: We have two location for 1940. You have therefore to manage this case: identify the attribute indicating this fact and write a code to collect those lines and insert in the previous table in the right place**

* Collect headers of the HTML table
* Build a data frame from the result table and the headers








In [ ]:
import re
# recupere le tableau des jeux olympiques
table = page.find("table", attrs="wikitable center")

# recupere le titre du tableau
tab_title = table.a.text

def processing(line) : 
    line = re.sub(r"\[.*\]|\(.*\)", "", line)
    line = line.split("\n")
    line = [x for x in line if x != ""]

    return line


def find_nb_ligne(tr):
    rows = 0
    td = tr.find("td")
    if td is not None and td.get('rowspan') is not None:
        rows = td.get('rowspan')
    return rows


# recupere les lignes du tableau et les stocker dans une liste
list_ligne = []
nb_rows = -1
entete = []
for ligne in table.find_all("tr"):
    process_line = processing(ligne.text)

    if nb_rows != 0 and int(find_nb_ligne(ligne))  != 0:
        nb_rows = int(find_nb_ligne(ligne))
        entete = process_line[0:2]
        list_ligne.append(process_line)
        nb_rows -= 1
    else:
        if nb_rows > 0:
            process_line = entete + process_line
            list_ligne.append(process_line)
            nb_rows -= 1
        else:
            list_ligne.append(process_line)
    

#enteete du tableau
tab_header = list_ligne[0]
list_ligne



In [ ]:
print(tab_title)
df = pandas.DataFrame(list_ligne[1:], columns=tab_header)
df

## Exercice 2: Locate places on a map

The objective here is to identify organizer cities on a map.
You will have to code the following steps:
* Collect the URL of each city
* Go the this page using `urllib.request`
* Find the coordinates
* Store cities and coordinates in a data frame

In [ ]:
import requests
from lxml import html
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

response = requests.get(jo)
page_for_links = html.fromstring(response.content)

links = page_for_links.xpath('//table[@class="wikitable center" and caption[contains(a, "Jeux olympiques d\'été")]]//tr/td[3]//a/@href')
links = ["https://fr.wikipedia.org/" + x for x in links]
print (links)


In [ ]:
Coordonnees = []
for link in links:
    response = requests.get(link)
    page_for_links = html.fromstring(response.content)
    coord = page_for_links.xpath('//tr[th[contains(a, "Coordonnées")]]/td/a/text()')
    coord_clean = [re.sub('\xa0', ' ', c) for c in coord]
    Coordonnees.append(coord_clean)
Coordonnees



In [ ]:
for i , liste in enumerate(Coordonnees) : 
    if liste == [] : 
        Coordonnees[i] = ["NaN"]
Coordonnees

In [ ]:
# Aplatir la liste et marquer les listes vides comme "vide"
Coordonnees_flat = [item if len(sublist) != 0 else "vide" for sublist in Coordonnees for item in sublist] + ["vide", "vide"]



df["Coordonnees"] = Coordonnees_flat
df

We then transform degree coordinates into numerical coordinates to build a map


In [ ]:
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S', 'O'):
        dd *= -1
    return dd

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    #potentially needs an adaptation
    #[student]
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
    #lng = dms2dd(parts[4], parts[5], parts[6], parts[7])
    return lat

data['latitude'] = data['latitude'].apply(parse_dms)

print(data)

The map can be obtain with the following code

In [ ]:
import geopandas as gpd
from pathlib import Path
import folium
from IPython.display import display

gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

Path("leaflet").mkdir(parents=True, exist_ok=True)

center = gdf[['latitude', 'longitude']].mean().values.tolist()
sw = gdf[['latitude', 'longitude']].min().values.tolist()
ne = gdf[['latitude', 'longitude']].max().values.tolist()

m = folium.Map(location = center, tiles='openstreetmap')

# I can add marker one by one on the map
for i in range(0,len(gdf)):
    folium.Marker([gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude']], popup=gdf.iloc[i]['name']).add_to(m)

m.fit_bounds([sw, ne])
display(m)

# Project

The objective of this project is to practice all concepts taught in the main lecture. Therefore, you will have to collect data around a thematic, identify a problematic, clean and format the data, provide some exploratory analysis and visualization, build models and evaluate them. You will have also to design dashboard and to storytell the whole pipeline.


The whole project will have two outputs describing the methodology and the results:
* A technical report targeting your datascientist colleagues
* An oral presentation targeting your CEO, chief, client. In this case, we assume that the audience is not specialized in data science. (But you also need to present the methodology to convince them to trust the results).


**Requests:**
* Team work of two people (same group throughout the semester)
* All your work should be stored on a git repo: [tutorial](https://github.com/baskiotisn/2IN013robot2023/blob/d979333fb80c9b6acd9515aaec040943d10d365c/docs/tutoriel_git.pdf)

**Remarks:**
There are also other libraries or issues you'll encounter when collecting data. Some of these are listed below, but don't be shy and interact with a search engine to solve your own issue!
* [Regular expressions](https://docs.python.org/3/howto/regex.html) might be useful!
* [API with authentication](https://www.geeksforgeeks.org/authentication-using-python-requests/)
*   [Selenium](https://selenium-python.readthedocs.io/) when pages are generated wia javascript scripts
* [Playwright](https://playwright.dev/) --> looks easier and more adapted than Selenium
* [Scrapy](https://scrapy.org/) for web crawling / or when you don't know the URL.[Tuto here](https://doc.scrapy.org/en/latest/intro/tutorial.html)
* [Summary of some difficulties in scrapping data from the web](https://www.zenrows.com/blog/web-scraping-challenges#page-structure-changes)

## Your daily task
Collecting data from the web and open data portals. Open data might include csv files, you can use them, but be aware that scraping should be your main acitivity in the dataset gathering.
You can begin to format the data, merge them to build dataframe that would be analyzed next week.